# Bag of Words Meets Bags of Popcorn

- <https://www.kaggle.com/c/word2vec-nlp-tutorial/data>

## 자연 언어 처리 - 위키백과, 우리모두의 백과사전
 - 자연어 처리는 인간이 발화하는 언어현상을 기계적으로 분석해서 컴퓨터가 이해할 수 있는 형태로 만드는 자연 언어 이해 또는 그러한 형태를 다시 인간이 이해할 수 있는 언어로 표현하는 제반 기술을 의미 (출처 : 위키피디아)

### 파트 1
 - 초보자를 대상으로 기본 자연어 처리를 다룬다.
 
### 파트 2, 3
 - word2vec을 사용하여 모델을 학습시키는 방법과 감정분석에 단어 벡터를 사용하는 방법을 본다.
 - 파트 3에서는 레시피를 제공하지 않고 word2vec을 사용하는 몇 가지 방법을 실험.
 - K-means 알고리즘을 사용해 군집화 해본다.
 - 긍정/부정 리뷰가 섞여 있는 100000만 개의 IMDB 감정분석 데이터 세트를 통해 목표를 달성해 본다.
 
 ### BOW(bag of words)
 - 가장 간단하지만, 효과적이라 널리 쓰이는 방법
 - 장, 문단, 문장, 서식과 같은 입력 텍스트의 구조를 제외하고 각 단어가 이 말뭉치에 얼마나 많이 나타나는지만 헤아린다.
 - 구조와 상관없이 단어의 출현횟수만 세기 때문에 텍스트를 담는 가방(bag)으로 생각할 수 있다. 
 - BOW는 단어의 순서가 완전히 무시 된다는 단점. 
   > - it's bad, not good at all.
   > - It's good, not bad at all.
 - 위 두문장은 의미가 전혀 반대지만 완전히 동일하게 반환된다.
 * 이를 보완하기 위해 n-gram을 사용하는데 BOW는 하나의 토큰을 사용하지만, n-gram은 n개의 토큰을 사용할 수 있도록 한다.

### 시작하기 전에
 NLP 연습

In [3]:
import pandas as pd

"""
header = 0 은 파일의 첫 번째 줄에 열 이름이 있음을 나타내며
delimiter = \t 는 필드가 탭으로 구분되는 것을 의미
quoting = 3은 큰 따옴표를 무시하도록 한다.
"""

# quote_minimal (0), qoute_all(1), quote_nonnumeric(2) or quote_none (3).

#레이블인 sentiment가 있는 학습 데이터
train = pd.read_csv("C://BigData/Kaggle/BoWMBoP/labeledTrainData.tsv", header=0, delimiter='\t',quoting=3)

#레이블이 없는 테스터 데이터
test = pd.read_csv("C://BigData/Kaggle/BoWMBoP/testData.tsv", header=0, delimiter='\t', quoting=3)

train.shape

(25000, 3)

In [4]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [5]:
test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [6]:
train.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

In [7]:
test.columns.values #레이블인 'sentiment'가 없다. 이 데이터를 기계학습을 통해 예측한다.

array(['id', 'review'], dtype=object)

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


In [9]:
train.describe()

,sentiment
count,25000.00000
mean,0.50000
std,0.50001
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


In [12]:
train['sentiment'].value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [13]:
# html 태그가 섞여 있기 때문에 이를 정제해줄 필요가 있음
train['review'][0][:700]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely lik'